In [2]:
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import numpy as np
from pickle import load
import joblib
from keras.models import load_model
import requests
from datetime import datetime

In [89]:
def threat():
  api_url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key=5Bo8lnm4ixoxdwcgqI2g4JDZt905zJFh&point=12.9716,77.5946"
  response = requests.get(api_url)
  response=response.json()
  with open('/content/Traffic_prophet_1.json', 'r') as fin:
    m = model_from_json(fin.read())
  future = m.make_future_dataframe(periods=1)
  future=future.tail(1)
  future['Avgspeed']=response['flowSegmentData']['currentSpeed']/response['flowSegmentData']['freeFlowSpeed']
  forecast = m.predict(future)
  preds_1=forecast['yhat']
  loaded_rf = joblib.load("/content/weather.joblib")
  now = datetime.now()
  current_time = now.strftime("%Y-%m-%d %H:%M:%S")
  k=current_time.split(' ')[0]+'T'+current_time.split(' ')[1]
  api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London,UK/"+k+"?key=73AZ2MH43JPSHUFG5VLKGUPFF&include=current"
  response = requests.get(api_url)
  response=response.json()
  a=np.array([response['days'][0]['humidity'],response['days'][0]['precip'],response['days'][0]['windspeed']])
  preds_2=loaded_rf.predict([a])
  scaler_weather = load(open('/content/scaler_weather.pkl', 'rb'))
  preds_2=scaler_weather.transform(preds_2.reshape(-1, 1))
  Threat_model=load_model("/content/score_model (1).h5")
  k=Threat_model.predict([preds_1,preds_2],verbose=0)
  return k[0][0]





















In [90]:
threat()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


0.13241814